### 1. 환경설정

In [1]:
# ✅ (필수) 기본 라이브러리 임포트
import torch  # CUDA/장치 정보 확인 및 dtype 설정을 위해 사용
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

c:\requirement\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. CUDA 점검

In [2]:
# ✅ CUDA 사용 가능 여부와 장치 이름 출력(디버깅용)
print("CUDA 사용 가능:", torch.cuda.is_available())  # True면 GPU 사용 가능
if torch.cuda.is_available():  # GPU가 있을 때만 장치명 출력
    print("GPU 이름:", torch.cuda.get_device_name(0))  # 첫 번째 GPU 이름


CUDA 사용 가능: True
GPU 이름: NVIDIA GeForce RTX 4060 Laptop GPU


### 3. 수식 테스트

In [21]:
# 1) 로컬 모델/토크나이저 로드 -------------------------------------------------
MODEL_ID = "Qwen/Qwen2.5-Math-1.5B"

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    use_fast=True,
    trust_remote_code=True
)

In [22]:
# device_map 제거, dtype로 FP16 지정
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,   # dtype → torch_dtype 로 권장
    device_map="auto",           # GPU 자동 매핑
    low_cpu_mem_usage=True,
    trust_remote_code=True
)

In [23]:
# GPU로 직접 이동 (accelerate 불필요)
model.to("cuda")

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [24]:
# pad/eos 정합성
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id

In [25]:
# 2) 파이프라인 구성 (model이 이미 cuda에 올라가 있으므로 device 지정 불필요) ---
generator = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True
)

Device set to use cuda:0


In [26]:
# 3) 수식(LaTeX) ----------------------------------------------------------------
EQN = r"""
\begin{align}
    \max_{\Theta} \sum_{(x,y)\in\cZ}  \sum_{t=1}^{|y|}  \log\left({p_{\Phi_0+\Delta\Phi(\Theta)}(y_{t} | x, y_{<t}})\right)
\label{eq:ft_add}
\end{align}
"""

In [27]:
# 4) 시스템/유저 메시지 ----------------------------
system_msg = (
    "You are a math tutor. Explain in clear, simple English so that a middle-school student can understand. "
    "Do NOT provide final numeric answers; focus only on interpretation and intuition. "
    "Keep all symbols exactly as written (same letters, indices i,j, hats, and √); "
    "do not introduce new symbols; do not expand (a-b)^2; "
    "do not restate the prompt or echo the equation unless strictly necessary."
)

user_msg = f"다음 수식을 중학생도 이해할 수 있도록 쉽게 설명합니다.\n\n[수식]\n{EQN}"

In [28]:
# 5) Chat 템플릿 적용 -----------------------------------------------------------
chat_prompt = tokenizer.apply_chat_template(
    [{"role": "system", "content": system_msg},
     {"role": "user", "content": user_msg}],
    tokenize=False,
    add_generation_prompt=True
)

In [29]:
# 6) 생성 실행(결정적; temperature 미사용 → 경고 없음) ---------------------------
gen_kwargs = {
    "max_new_tokens": 700,
    "do_sample": False,
    "no_repeat_ngram_size": 4,
    "repetition_penalty": 1.1,
    "return_full_text": False,
    "eos_token_id": tokenizer.eos_token_id
}
out = generator(chat_prompt, **gen_kwargs)

In [30]:
# 7) 결과 출력 -------------------------------------------------------------------
print("=== 모델 출력 ===\n")
print(out[0]["generated_text"])

=== 모델 출력 ===

This is the formula you provided:

\[
\max_{\theta} \sum{(x,y) \in \mathcal{Z}} \sum_{i=1}^{n} \log\big(p_{\phi_0 + \delta\phi(\theta)}(y_i | x, \mathbf{y}_{<i})\big)
\]

Let's break it down step by step to make it easier for a middle school student to understand.

### Step 1: Understanding the Summation Notation

The symbol \(\sum\) means "sum." The expression inside the summation tells us what we are adding up.

- \((x, y) \in Z\): This means we are considering pairs of \(x\) and \(y\) from a set called \(Z\).
- \(\sum_{i = 1}^{N}\): This means summing over all values of \(i\) from 1 to \(N\).

So, the first part of the formula is:
\[
\sum_{(X,Y) \in X} \sum_i \log\Bigg(p_{\Phi_{0} + \Delta\Phi (\Theta)}(Y_i | X, Y_{<i}) \Bigg)
\]

### Step 2: Breaking Down the Inner Summation

The inner summation \(\sum_i\) means we are summing over each element \(i\) in the sequence \(Y\).

- \(Y_i\): This is the \(i\)-th element of the sequence \(y\).
- \(X\): This represents the in